# 1. Packages Requirement

- `--no-deps`: Không cài đặt tự động các phụ thuộc của các gói này, tránh xung đột phiên bản trong Colab (vì Colab đã có một số thư viện cài sẵn).

- `bitsandbytes`: Thư viện hỗ trợ lượng tử hóa (quantization) như 4-bit, 8-bit, giúp giảm dung lượng bộ nhớ khi chạy mô hình lớn.

- `accelerate`: Thư viện của Hugging Face để tăng tốc huấn luyện và inference trên nhiều thiết bị (CPU, GPU, TPU).

- `xformers==0.0.29`: Thư viện tối ưu hóa attention trong Transformer, cải thiện tốc độ và giảm bộ nhớ. Chỉ định phiên bản 0.0.29 để đảm bảo tương thích.

- `peft`: Thư viện Parameter-Efficient Fine-Tuning của Hugging Face, hỗ trợ các kỹ thuật như LoRA/QLoRA mà Unsloth sử dụng.

- `trl`: Thư viện Transformers Reinforcement Learning, hỗ trợ huấn luyện mô hình với các phương pháp như RLHF (Reinforcement Learning from Human Feedback).

- `triton`: Thư viện từ OpenAI để tối ưu hóa kernel GPU, tăng tốc tính toán trong PyTorch.

- `cut_cross_entropy`: Một gói tối ưu hóa hàm mất mát cross-entropy, thường được dùng để tăng tốc huấn luyện mô hình ngôn ngữ.

- `unsloth_zoo`: Một gói phụ của Unsloth, cung cấp các mô hình đã được tối ưu hóa hoặc các công cụ bổ sung để làm việc với Unsloth.

- `sentencepiece`: Thư viện mã hóa văn bản (tokenization), thường dùng cho các mô hình như Llama hoặc DeepSeek.

- `protobuf`: Thư viện Google Protocol Buffers, cần thiết để làm việc với định dạng dữ liệu trong một số mô hình hoặc công cụ Hugging Face.

- `datasets`: Thư viện của Hugging Face để tải và xử lý tập dữ liệu huấn luyện/inference.

- `huggingface_hub`: Thư viện để tương tác với Hugging Face Hub (tải mô hình, dataset, đẩy kết quả lên Hub).

- `hf_transfer`: Công cụ tăng tốc tải xuống từ Hugging Face Hub, hữu ích khi tải các mô hình lớn.



In [ ]:
# # %%capture
# import os
# if "COLAB_" not in "".join(os.environ.keys()):
#     !pip install unsloth
# else:
#     # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
#     !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
#     !pip install --no-deps cut_cross_entropy unsloth_zoo
#     !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
#     !pip install --no-deps unsloth

# 2. Config LLM Model

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
print(torch.cuda.get_device_capability())


True
NVIDIA GeForce RTX 5080
(12, 0)


In [ ]:
# import os

# os.environ["TORCH_CUDA_ARCH_LIST"] = "12.0"

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/tyan/Simple-Fine-Tune-Llama/finetune_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.5: Fast Llama patching. Transformers: 4.53.2.
   \\   /|    NVIDIA GeForce RTX 5080. Num GPUs = 1. Max memory: 15.92 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# 3. LoRA Adapters

In [ ]:
# Parameter-Efficient Fine-Tuning (PEFT)
# LoRA (Low-Rank Adaptation)
model = FastLanguageModel.get_peft_model(
    model,    # pre-trained model
    r = 32,
    # q_proj, k_proj, v_proj: Các lớp tạo Query, Key, Value trong cơ chế Attention.
    # o_proj: Lớp chiếu đầu ra của Attention.
    # gate_proj, up_proj, down_proj: Các lớp trong mạng Feed-Forward (FFN)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    # Các ma trận Low-Rank (A.B) được nhân với lora_alpha/r để điều chỉnh mức độ ảnh hưởng của chúng lên trọng số gốc.
    lora_alpha = 32,

    # Dropout là kỹ thuật regularization, ngẫu nhiên bỏ qua một tỷ lệ đơn vị để tránh overfitting.
    lora_dropout = 0,

    # Quy định cách xử lý bias (độ lệch) trong các lớp LoRA.
    bias = "none",

    # Kích hoạt gradient checkpointing để tiết kiệm bộ nhớ khi huấn luyện.
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,

    # RS-LoRA là một biến thể của LoRA, cải thiện tính ổn định khi dùng rank cao.
    # Nếu bạn tăng r và gặp vấn đề ổn định, có thể thử bật True.
    use_rslora = False,

    # LoFTQ kết hợp quantization với LoRA để nén mô hình thêm.
    loftq_config = None,
)

Unsloth 2025.7.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


# 4. Format Prompting

In [ ]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:            
# Company database: {}

# ### Input:
# SQL Prompt: {}

# ### Response:
# SQL: {}

# Explanation: {}
# """

# # End Of Sequence Token
# EOS_TOKEN = tokenizer.eos_token

# def formatting_prompts_func(examples):
#   company_databases = examples["sql_context"]
#   prompts = examples["sql_prompt"]
#   sqls = examples["sql"]
#   explanations = examples["sql_explanation"]
#   texts = []

#   for company_database, prompt, sql, explanation in zip(company_databases, prompts, sqls, explanations):
#     # Must add EOS_TOKEN, otherwise your generation wil go on forever!
#     # .format replace `{}`
#     text = alpaca_prompt.format(company_database, prompt, sql, explanation) + EOS_TOKEN
#     texts.append(text)

#   return {"text" : texts, }

# pass

In [ ]:
from huggingface_hub import login
from transformers import AutoTokenizer

login(token="")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [ ]:
# End Of Sequence Token (rất quan trọng)
EOS_TOKEN = tokenizer.eos_token

def formatting_qa_func(examples):
    all_texts = []
    for messages in examples["messages"]:
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        text += EOS_TOKEN
        all_texts.append(text)
        
    return {"text": all_texts}

        

In [ ]:
from datasets import load_dataset


dataset = load_dataset("json", data_files="./data/data.jsonl", split="train")
dataset = dataset.map(formatting_qa_func, batched=True)

dataset[0]

{'messages': [{'role': 'user', 'content': 'Tên của khách sạn là gì?'},
  {'role': 'assistant', 'content': 'Tên khách sạn là Simon Hotel.'}],
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Jul 2025\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTên của khách sạn là gì?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTên khách sạn là Simon Hotel.<|eot_id|><|eot_id|>'}

# 5. SFTTrainer (Supervised Fine-Tuning Trainer)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset=None,
    max_seq_length = max_seq_length,
    # Số lượng tiến trình song song để xử lý dữ liệu.
    dataset_num_proc = 1,

    # Nếu True, các mẫu ngắn sẽ được gộp lại để tận dụng tối đa max_seq_length, tăng hiệu quả huấn luyện.
    packing = False,

    args = TrainingArguments(
        # Kích thước batch trên mỗi thiết bị (GPU/CPU).
        per_device_train_batch_size = 4,

        # Số bước tích lũy gradient trước khi cập nhật trọng số.
        gradient_accumulation_steps = 4,

        # Số bước khởi động để tăng dần learning rate.
        warmup_steps = 5,
        # num_train_epochs = 1,     # Set this for full training run.
        max_steps = 60,
        learning_rate = 2e-4,

        # Nếu không hỗ trợ BF16 (16-bit floating point), FP16 được dùng để giảm bộ nhớ và tăng tốc, nhưng có thể giảm độ chính xác một chút.
        #fp16 = is_bfloat16_supported(),

        # BF16 giữ phạm vi số lớn hơn FP16, tốt hơn cho huấn luyện mô hình lớn. Nếu phần cứng hỗ trợ (như GPU NVIDIA Ampere trở lên), BF16 được ưu tiên.
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,

        # Bộ tối ưu hóa (optimizer) được sử dụng.
        optim = "adamw_8bit",

        # Hệ số điều chỉnh suy giảm trọng số (regularization).
        weight_decay = 0.01,

        # Learning rate giảm dần tuyến tính từ giá trị tối đa (2e-4) về 0 qua 60 bước, giúp mô hình hội tụ tốt hơn vào cuối.
        lr_scheduler_type = "linear",

        # random seed
        seed = 3047,
        output_dir = "outputs",
    ),
)

: 

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 37 | Num Epochs = 20 | Total steps = 60
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 22,544,384 of 1,258,358,784 (1.79% trained)
CUDA error (/__w/xformers/xformers/third_party/flash-attention/hopper/flash_bwd_launch_template.h:219): invalid argument


In [ ]:
# model.save_pretrained_gguf("llama3-sql", tokenizer, quantization_method="f16")

In [ ]:
# !zip -r /content/llama3-sql.zip /content/llama3-sql